# Технологии обработки больших данных

Занятие 3. PySpark Data Structures

0. Запуск PySpark на локальной машине
1. Spark DataFrame 
2. Spark RDD (разбор предыдущего ДЗ)
3. Spark Pandas API DataFrame
4. Домашнее задание 


## 0. Запуск PySpark на локальной машине

При первом запуске spark на локальной машине нужно установить следующие зависимости:  

- Java 8 
- Apache Spark 3.2.1 with hadoop 3.2, 
- Findspark to locate the spark in the system. 

Если работаете на локальной машине и зависимости уже установлены, эту ячейку запускать не нужно.

In [ ]:
%%bash
apt-get install openjdk-8-jdk-headless -qq > /dev/null
wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
tar xvf spark-3.2.1-bin-hadoop3.2.tgz > /dev/null
pip install -q findspark

Зависимости для работы Pandas API

In [ ]:
%%bash
pip install pandas pyarrow plotly

Начните с этой ячейки

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = os.getcwd() + "/spark-3.2.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Рассмотрим пример данных [German Credit](https://www.kaggle.com/uciml/german-credit), которые используются для решении задачи кредитного скоринга. Это небольшой датасет с информацией о клиентах, необходимой для принятия решения - выдавать кредит или нет.  

Сегодня мы не будем решать задачу предсказания, просто разберемся с основными приемами EDA (Exploratory data analysis, [Разведочный анализ данных](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%B7%D0%B2%D0%B5%D0%B4%D0%BE%D1%87%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85)). 

In [ ]:
DATA_PATH = 'sample_data/credit_data.csv'

**Columns**  

Age (numeric)  
Sex (text: male, female)  
Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)  
Housing (text: own, rent, or free)  
Saving accounts (text - little, moderate, quite rich, rich)  
Checking account (numeric, in DM - Deutsch Mark)  
Credit amount (numeric, in DM)  
Duration (numeric, in month)  
Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

## 1. Spark DataFrame

Базовый класс для работы со структуированными данными в pyspark.

In [ ]:
df = spark.read.csv(DATA_PATH, header=True)
type(df)

In [ ]:
# First rows in this DataFrame
df.show(10, truncate=False) 

### Схема данных как в SQL

In [ ]:
schema = "id INT, Age INT, Sex STRING, Job INT, Housing STRING, Saving_accounts STRING, \
Checking_account STRING, Credit_amount INT, Duration INT, Purpose STRING"

In [ ]:
df = spark.read.csv('sample_data/credit_data.csv', schema=schema, header=True )

In [ ]:
df.printSchema()

### Сортировка и фильтрация данных

In [ ]:
# One column sorting
df.sort('Job', ascending=False).show()

In [ ]:
# Few columns sorting
df.sort(['Age', 'Credit_amount'], ascending=[False, True]).show()

In [ ]:
df_car = df.filter(df["Purpose"] == 'car')

In [ ]:
# Доля автокредитов
df_car.count() / df.count()

### Группировка данных

In [ ]:
df.groupBy("Age").count().sort('Age').show()

## 2. Spark RDD 

Resilient Distributed Dataset. 
Менее удобный, но более производительный контейнер для данных.  

Подробнее про DataFrame, DataSet и RDD на русском языке
[1](https://www.bigdataschool.ru/blog/spark-sql-data-structures.html), 
[2](https://www.bigdataschool.ru/blog/rdd-vs-dataframe-vs-dataset.html).  

На английском рекомендую [официальный гайд](https://spark.apache.org/docs/latest/sql-getting-started.html).

In [ ]:
log_file = spark.read.text('log.txt')

In [ ]:
%%time

# Note, we cant use   lambda x:   x.value.upper()
df = log_file.rdd.map(lambda x: ( x.value.upper() ,) ).toDF() 

df.show(truncate=False)

In [ ]:
with open('white_list.txt') as f:
    ww = f.readlines()

ww = "".join([w for w in ww]).split()
ww = list(map(str.lower, ww))

In [ ]:
def white_filter(s):
    w = s.value.split()
    timestamp = str(w[0]) + str(w[1]) + " "
    words = w[2:]
    filtered_words = list(filter(lambda x: x in ww, words))
    return (timestamp, " ".join([w for w in filtered_words])  )

In [ ]:
%%time
df2 = log_file.rdd.map(white_filter).toDF(schema=('TimeStamp', 'Words'))
df2.show(truncate=False)

In [ ]:
wordCounts = df2.rdd.flatMap(lambda line: line[1].split(" "))\
                      .map(lambda word: (word, 1))\
                      .reduceByKey(lambda a, b: a + b)

In [ ]:
%%time
wordCounts.toDF().show()

## 3. Spark Pandas API 

Начиная с версии Spark 3.2 имеется реализация Pandas API.    
Хороший материал непосредственно по pandas: [mlcourse.ai](https://habr.com/ru/company/ods/blog/322626/)

In [ ]:
# import pandas as pd

import pyspark.pandas as pd

In [ ]:
df = pd.read_csv('sample_data/credit_data.csv')
type(df)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Средний возраст заемщиков 
df['Age'].mean()

In [ ]:
# Статистика по всем числовым колонкам
df.describe()

### Индексация и фильтрация данных

In [ ]:
# Индексация python slices 
df[1:11:2]

In [ ]:
# Фильтрация по условию
df[df["Sex"] == 'male']

In [ ]:
# Какой средний размер кредита у заемщиков мужчин?
df[df["Sex"] == 'male']['Credit_amount'].mean()

In [ ]:
# Какой средний размер кредита у заемщиков женщин?
df[df["Sex"] == 'female']['Credit_amount'].mean()

### Группировка данных

In [ ]:
# Группировка разделяет df на несколько частей, в которых значения заданной колонки будут одинаковыми
df.groupby('Sex')

In [ ]:
# Можно указать какие колонки нас интересуют 
df.groupby('Sex')['Credit_amount']

In [ ]:
# В конце группировки нужно указать функцию
df.groupby('Sex')['Credit_amount'].mean()

### Визуализация

In [ ]:
df['Age'].hist()

## 4. Домашнее задание  



1. Сколько мужчин и женщин (признак Sex) представлено в этом наборе данных?

In [ ]:
# Ваш код здесь

2. Каков средний возраст (признак Age) женщин?

In [ ]:
# Ваш код здесь

3. Какова доля заемщиков с собственным жильем (признак Housing)?

In [ ]:
# Ваш код здесь

4. Каково среднее значение возраста тех, кто имеет высокие накопления (признак Saving_accounts)?

In [ ]:
# Ваш код здесь

5. Каково среднеквадратичное отклонения возраста тех, кто имеет высокие накопления (признак Saving_accounts)?

In [ ]:
# Ваш код здесь

6. Выведите гистограмму категорий покупок (признак Purpose) для мужчин и женщин.

In [ ]:
# Ваш код здесь

7. На что чаще всего берутся длинные кредиты (более 24 мес)?

In [ ]:
# Ваш код здесь

8. Какой средний срок кредита (признак Duration) для заемщиков, имеющих высокие текущие траты (признак Checking_account)?

In [ ]:
# Ваш код здесь

9. Какой средний срок кредита (признак Duration) для заемщиков, имеющих низкие текущие траты (признак Checking_account)?

In [ ]:
# Ваш код здесь

10. На какую цель взят самый дорогой кредит?

In [ ]:
# Ваш код здесь